# Generate shell scripts

## Set up parameters to run

In [15]:
# Some parameters

layer = 4  # in {2, 3, 4} default 4, usually only care about 4 hid layer network
flame_ann = "flame2_ann3"

n_epoch = 500
save_dir = "/home/xux/work/Flame_ann/"
max_runtime = "4:00:00"
split_size = 16

layer1 = [25, 50, 100, 200]
layer2 = [25, 50, 100, 200]
layer3 = [25, 50, 100, 200]
layer4 = [25, 50, 100, 200]

## Generate commands based on number of layers

In [16]:
def get_2l_commands(layer1, layer2):
    commands = []
    for i2 in range(len(layer2)):
        for i1 in range(len(layer1)):
            commands.append(f"python {flame_ann}_cpu.py --data_dir $data_dir --epochs "
                            + f"$epochs --nodes {layer1[i1]} {layer2[i2]} "
                            + f">{flame_ann}_2l-{layer1[i1]}_{layer2[i2]}.out\n")
    return commands


def get_3l_commands(layer1, layer2, layer3):
    commands = []
    for i3 in range(len(layer3)):
        for i2 in range(len(layer2)):
            for i1 in range(len(layer1)):
                commands.append(f"python {flame_ann}_cpu.py --data_dir $data_dir --epochs "
                                + f"$epochs --nodes {layer1[i1]} {layer2[i2]} {layer3[i3]} "
                                + f">{flame_ann}_3l-{layer1[i1]}_{layer2[i2]}_{layer3[i3]}.out\n")
    return commands


def get_4l_commands(layer1, layer2, layer3, layer4):
    commands = []
    for i4 in range(len(layer4)):
        for i3 in range(len(layer3)):
            for i2 in range(len(layer2)):
                for i1 in range(len(layer1)):
                    commands.append(f"python {flame_ann}_cpu.py --data_dir $data_dir --epochs "
                                    + f"$epochs --nodes {layer1[i1]} {layer2[i2]} {layer3[i3]} {layer4[i4]} "
                                    + f">{flame_ann}_4l-{layer1[i1]}_{layer2[i2]}_{layer3[i3]}_{layer4[i4]}.out\n")
    return commands

In [17]:
if layer == 2:
    commands = get_2l_commands(layer1, layer2)
elif layer == 3:
    commands = get_3l_commands(layer1, layer2, layer3)
elif layer == 4:
    commands = get_4l_commands(layer1, layer2, layer3, layer4)
else:
    assert "Error in num of layers"

In [18]:
len(commands)

256

## Write commands to sbatch files

### Define sbatch prefix and suffix

Prefix contents

```shell
#!/bin/bash
#SBATCH --time=4:00:00
#SBATCH --cpus-per-task=60
#SBATCH --mem=64G
#SBATCH --partition=batch
#SBATCH --job-name=2layer
#SBATCH --output=./%x-%j-slurm.out
#SBATCH --error=./%x-%j-slurm.err

start_time=$(date +"%T")
echo "Start time: $start_time"

data_dir=/home/xux/work/Flame_ann/data/
epochs=100
```

Suffix contents
```shell
finish_time=$(date +"%T")
echo "Finish time: $finish_time"
```

### for err checking
```python
file = open("test_commands.txt", "w")
[file.write(cmd) for cmd in commands]
file.close()

n_cmd = len(commands)
n_full_splits = n_cmd // split_size
print(split_size)
for spl in range(n_full_splits):
    print(spl)
    [print(cmd) for cmd in commands[spl*split_size:(spl+1)*split_size]]
```

In [19]:
# define useful prefix and suffix
def get_sbatch_prefix(split):
    sbatch_prefix = f"#!/bin/bash\n#SBATCH --time={max_runtime}\n#SBATCH --cpus-per-task=60\n#SBATCH --mem=64G\n#SBATCH --partition=batch\n#SBATCH --job-name={flame_ann}_{layer}layer_{split}\n#SBATCH --output=./%x-%j-slurm.out\n#SBATCH --error=./%x-%j-slurm.err\n"
    return sbatch_prefix


def get_sbatch_path(split):
    sbatch_path = f"./run_{flame_ann}_tune_{layer}layer_{split}.sbatch"  # f"./tune_model/run_{flame_ann}_tune_{layer}layer_{split}.sbatch"
    return sbatch_path


starttime_prefix = '\nstart_time=$(date +"%T")\necho "Start time: $start_time"\n'
data_epochs_prefix = f"\ndata_dir={save_dir}data/\nepochs={n_epoch}\n\n"
endtime_suffix = '\nfinish_time=$(date +"%T")\necho "Finish time: $finish_time"\n'

In [20]:
def write_commands(commands, split_size=100):
    n_cmd = len(commands)
    n_full_splits = n_cmd // split_size
    for spl in range(n_full_splits):
        file = open(get_sbatch_path(spl), "w")
        file.write(get_sbatch_prefix(spl))
        file.write(starttime_prefix)
        file.write(data_epochs_prefix)
        [file.write(cmd) for cmd in commands[spl*split_size:(spl+1)*split_size]]
        file.write(endtime_suffix)
        file.close()

    if n_cmd > n_full_splits*split_size:
        file = open(get_sbatch_path(n_full_splits), "w")

        file.write(get_sbatch_prefix(n_full_splits))
        file.write(starttime_prefix)
        file.write(data_epochs_prefix)

        [file.write(cmd) for cmd in commands[n_full_splits*split_size:-1]]

        file.write(endtime_suffix)
        file.close()


In [21]:
write_commands(commands, split_size)